# 0. 引入依赖包
注意 cx_Oracle裤需要动态依赖

In [5]:
import pandas as pd
import networkx as nx
import numpy as np
import os
import re
from db_con.pcn_oracle_data import PCNDB
from tqdm import tqdm
os.getcwd()

'/home/wzk/Code/PCN_ROUTES'

In [6]:
os.environ['LD_LIBRARY_PATH']

':/usr/local/cuda/lib64:/home/wzk/Oracle/instantclient_19_15'

# 1. 连接数据库

In [7]:
pcnTestDB = PCNDB('PCN_TEST','784427618', '10.112.199.111:1521/LHR11G')
bus_inf = pcnTestDB.get_all_businesses()
cha_inf = pcnTestDB.get_all_channels()

# 2. 获取有效通路
数据处理

In [8]:
def get_stn_name(stn_id, pcnDB, stnDict):
    del_full_name = lambda x:x[0].split('/')[-1] if x[0] != None else None
    if stn_id not in stnDict.keys():
        stn_dict = pcnDB.get_station_inf(stn_id, tCols=['FULL_NAME'])
        stn_name = del_full_name(stn_dict['FULL_NAME']) if stn_dict['FULL_NAME'] != None else None
        stnDict[stn_id] = stn_name
    else:
        stn_name = stnDict[stn_id]
    
    return stn_name, stnDict


def ch_channel_name(channelInf, pcnDB):
    assert 'A_STATION' in channelInf.keys() and 'Z_STATION' in channelInf.keys()
    stnList_a = channelInf['A_STATION']
    stnList_z = channelInf['Z_STATION']

    stnList_a_name , stnList_z_name = [], []
    resDict = {}

    for idx, (s_a, s_z) in enumerate(zip(stnList_a, stnList_z)):
        s_a_name, resDict = get_stn_name(s_a, pcnDB, resDict)
        s_z_name, resDict = get_stn_name(s_z, pcnDB, resDict)

        stnList_a_name.append(s_a_name)
        stnList_z_name.append(s_z_name)

    channelInf['A_STATION_NAME'] = stnList_a_name
    channelInf['Z_STATION_NAME'] = stnList_z_name

    return channelInf, resDict


def del_channel_inf(channelInf, pcnDB):
    channelInf, stnDict = ch_channel_name(channelInf, pcnDB)
    stnList_a = channelInf['A_STATION_NAME']
    stnList_z = channelInf['Z_STATION_NAME']

    assert len(stnList_a) == len(stnList_z)
    idx = 0
    while idx < len(stnList_a):
        s_a = stnList_a[idx]
        s_z = stnList_z[idx]
        if s_a is None or s_z is None:
            for k in channelInf.keys():
                channelInf[k].pop(idx)
        elif s_a == s_z:
            for k in channelInf.keys():
                channelInf[k].pop(idx)
        else:
            idx += 1

    channelInfPd = pd.DataFrame(data=channelInf)
    return channelInfPd, stnDict


def con_business_inf(b_id, pcnDB, stnDict):
    busInfDict = pcnDB.get_business_inf(b_id)
    stnID_a, stnID_z = busInfDict['A_SITE_ID'][0], busInfDict['Z_SITE_ID'][0]

    stnName_a = get_stn_name(stnID_a, pcnDB, stnDict)
    stnName_z = get_stn_name(stnID_z, pcnDB, stnDict)
    busInfDict['A_STATION_NAME'] = stnName_a
    busInfDict['Z_STATION_NAME'] = stnName_z

    return busInfDict


def con_business_stn_inf(b_id, pcnDB, stnDict):
    resDitc={}
    busInfDict = pcnDB.get_business_inf(b_id)
    stnID_a, stnID_z = busInfDict['A_SITE_ID'][0], busInfDict['Z_SITE_ID'][0]

    stnName_a,_ = get_stn_name(stnID_a, pcnDB, stnDict)
    stnName_z,_ = get_stn_name(stnID_z, pcnDB, stnDict)
    resDitc['A_STATION_NAME'] = stnName_a
    resDitc['Z_STATION_NAME'] = stnName_z

    return resDitc

 
def del_business_inf(businessInf, pcnDB, stnDict=None):
    assert 'BUSINESS_ID' in businessInf.keys() and 'CHANNEL_ID' in businessInf
    busDict = {}
    for idx, busID in enumerate(businessInf['BUSINESS_ID']):
        if busID not in busDict.keys():
            busDict[busID] = [businessInf['CHANNEL_ID'][idx]]
        else:
            busDict[busID].append(businessInf['CHANNEL_ID'][idx])

    businessInfDict ={}
    for idx, b_id in enumerate(list(busDict.keys())):
        busInfDict = con_business_stn_inf(b_id, pcnDB, stnDict)
        businessInfDict[b_id] = busInfDict

    return busDict, businessInfDict


def generate_pcnMap(channelInf):
    pcnMap = nx.Graph()
    stnList_a = channelInf['A_STATION_NAME']
    stnList_z = channelInf['Z_STATION_NAME']
    channelIdList = channelInf['OBJ_ID']

    edgeDict = {}
    for idx, (c_id, s_a, s_z) in enumerate(zip(channelIdList, stnList_a, stnList_z)):
        edgeStr = '{}-{}'.format(s_a, s_z)
        edgeStrRe = '{}-{}'.format(s_z, s_a)
        if edgeStr in edgeDict.keys():
            edgeDict[edgeStr].append(c_id)
        elif edgeStrRe in edgeDict.keys():
            edgeDict[edgeStrRe].append(c_id)
        else:
            pcnMap.add_edge(s_a, s_z)
            edgeDict[edgeStr] = [c_id]

    for e in pcnMap.edges.keys():
        edgekey = '{}-{}'.format(e[0], e[1])
        edgekeyRe = '{}-{}'.format(e[1], e[0])
        if edgekey in edgeDict.keys():
            pcnMap.edges[e]['channel_id'] = edgeDict[edgekey]
        else:
            pcnMap.edges[e]['channel_id'] = edgeDict[edgekeyRe]


    return pcnMap


def get_circuitous_bus(businessInfDict, pcnMap):
    busDelDict={}
    for idx,b_id in enumerate(businessInfDict.keys()):
        stnName_a = businessInfDict[b_id]['A_STATION_NAME']
        stnName_z = businessInfDict[b_id]['Z_STATION_NAME']

        if stnName_a not in pcnMap.nodes.keys() or stnName_z not in pcnMap.nodes.keys():
            continue
        elif stnName_a is None or stnName_z is None:
            continue
        elif stnName_a == stnName_z:
            continue
        else:
            busDelDict[b_id] = businessInfDict[b_id]

    return busDelDict

def get_circuitous_channel(busDelDict, busDict):
    chaDelDict={}
    for b_id in busDelDict.keys():
        if b_id in busDict.keys():
            chaDelDict[b_id] = busDict[b_id]
    
    return chaDelDict

def get_circuitous_inf(busInfDict, businessInfDict, pcnMap):
    pcnBussInf={}
    for _,b_id in enumerate(businessInfDict.keys()):
        stnName_a = businessInfDict[b_id]['A_STATION_NAME']
        stnName_z = businessInfDict[b_id]['Z_STATION_NAME']

        if stnName_a not in pcnMap.nodes.keys() or stnName_z not in pcnMap.nodes.keys():
            continue
        elif stnName_a is None or stnName_z is None:
            continue
        elif stnName_a == stnName_z:
            continue
        else:
            pcnBussInf[b_id] = businessInfDict[b_id]
    
    for b_id in pcnBussInf.keys():
        if b_id in busInfDict.keys():
            pcnBussInf[b_id]['CHANNELS'] = busInfDict[b_id]
    
    return pcnBussInf



In [9]:
channelInfPd, stnDict = del_channel_inf(cha_inf, pcnTestDB)
busInfDict, businessInfDict = del_business_inf(bus_inf, pcnTestDB, stnDict)
pcnMap = generate_pcnMap(channelInfPd)
# busDelDict = get_circuitous_bus(businessInfDict, pcnMap)
# chaDelDict = get_circuitous_channel(busDelDict, busInfDict)
pcnBussInf =  get_circuitous_inf(busInfDict, businessInfDict, pcnMap)

## 2.1 获取所有可用通路
利用nx.all_simple_paths

In [10]:
def get_access_routes(pcnBussInf, pcnMap, cutoff=3):
    resDict={}
    routeDict = {}
    busIDList = list(pcnBussInf.keys())
    for idx in tqdm(range(len(pcnBussInf))):
        b_id = busIDList[idx]
        stn_a, stn_z = pcnBussInf[b_id]['A_STATION_NAME'], pcnBussInf[b_id]['Z_STATION_NAME']
        resKey = "{}-{}".format(stn_a, stn_z)
        resKey1 = "{}-{}".format(stn_z, stn_a)
        if resKey in resDict.keys():
            routeDict[b_id] = resDict[resKey]
        elif resKey1 in resDict.keys():
            routeDict[b_id] = resDict[resKey1]
        else:
            nx_allRoutes = nx.all_simple_paths(pcnMap,source=stn_a,target=stn_z, cutoff=cutoff)
            routes = list(nx_allRoutes)
            routeDict[b_id] = routes
            resDict[resKey] = routes

    return routeDict
    
routeDict = get_access_routes(pcnBussInf, pcnMap)

100%|██████████| 18083/18083 [00:36<00:00, 494.87it/s]


# 3.路由优选

## 3.1 厂商筛选

In [11]:
def del_business_fac(chaList, pcnDB):
    assert isinstance(chaList, list)
    get_value = lambda x: x["NAME"][0] if 'NAME' in x.keys() else None

    facList = [get_value(pcnDB.get_fac_inf(c_id)) for c_id in chaList]
    facList = list(map(_del_fac_lable, facList))

    conDict = {}
    maxNum, maxName = 0, ''
    for f in facList:
        if f not in conDict.keys():
            conDict[f] = 1
        else:
            conDict[f] += 1
        if conDict[f] > maxNum:
            maxNum = conDict[f]
            maxName = f
    return maxName
    
def _del_fac_lable(facVal):
    if  not isinstance(facVal, str):
        return 'Others'
    key_words = ['华为', '依赛', '烽火', '马可尼', 'ECI北京', '中兴', '北电', '赛特']
    re_str = '|'.join(key_words)
    
    re_op_matchKeys = lambda x: [re.search(re_str, x), re.search('SDH', x)]
    re_op_delNone = lambda x: x.group(0) if x is not None else ''

    facValList = re_op_matchKeys(facVal)
    res_val = ''.join([re_op_delNone(i) for i in facValList])
    res_val = res_val if len(res_val) != 0 else facVal

    return res_val

def get_fac_inf(pcnBussInf, pcnDB):
    for _, busInfDict in pcnBussInf.items():
        fac = del_business_fac(busInfDict['CHANNELS'], pcnDB)
        busInfDict['FAC_LABLE'] = fac
    
    return pcnBussInf
    

In [12]:
pcnBussInf = get_fac_inf(pcnBussInf, pcnTestDB)

In [13]:
del_business_fac(pcnBussInf['648C6BD8-20AE-4B1B-88C6-F5064C708F28-77700']['CHANNELS'], pcnTestDB)

'华为SDH'

## 3.2 获取通道类型和业务类型

In [14]:
def get_business_type(pcnBussInf, pcnDB):
    for b_id, bussInfDict in pcnBussInf.items():
        b_type = pcnDB.get_buss_type(b_id)
        bussInfDict["B_TYPE"] = b_type['BUZ_TYPE'] if b_type['BUZ_TYPE'] != None else -1

    return pcnBussInf
    

In [15]:
def _del_cType(cList, channelInfPd):
    cTpyeList = []
    for cha in cList:
        res = channelInfPd.loc[channelInfPd['OBJ_ID'] == cha, "CHANNEL_TYPE"]
        cTpyeList.extend(res.to_list())
    return cTpyeList

def get_channel_type(pcnBussInf, channelInfPd):
    for b_id, bussInfDict in pcnBussInf.items():
        chaList = bussInfDict['CHANNELS']
        cTpyeList = _del_cType(chaList, channelInfPd)
        cType = cTpyeList[0] if len(cTpyeList) != 0 else -1
        bussInfDict['C_TYPE'] = cType

    return pcnBussInf
    

In [16]:
pcnBussInf = get_business_type(pcnBussInf, pcnTestDB)
pcnBussInf = get_channel_type(pcnBussInf, channelInfPd)

# -1.测试

In [4]:
pcnTestDB.get_buss_type(b_id='11')

{'BUZ_TYPE': [None]}

In [81]:
e_len_num, e_same_type = 0, 0
e_len_d, e_same_d ={}, {}
for b_id, d in pcnBussInf.items():
    c_type = d['C_TYPE']
    cList = d['CHANNELS']
    if len(cList) != len(c_type):
        e_len_num += 1
        e_len_d[b_id] = d
    for i in range(len(c_type)):
        if i == 0:
            continue
        else:
            if c_type[i-1] != c_type[i-1]:
                e_same_type += 1
                e_same_d[b_id] = d
                


print("Scan pcnBussInf:\n\t ERRO_LEN:{}\n\tERRO_TYPE:{}".format(e_len_num, e_same_type))


Scan pcnBussInf:
	 ERRO_LEN:1368
	ERRO_TYPE:0


In [ ]:
e_same_d

In [10]:
c, v = pcnTestDB._exec_sql("SELECT * FROM T_SPC_SITE WHERE OBJ_ID = 'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391'")

In [18]:
pcnDBInf={
        'USER': 'PCN_TEST',
        'PASS': '784427618',
        'SID': '172.17.0.2:1521/LHR11G'
    } 
print(" PCN DataBase INF ".center(50, '*'))
for k in pcnDBInf.keys():
    if k != 'PASS':
        print("\t* {}: {}".format(k, pcnDBInf[k]))
print(" Successfully connected! ".center(50, '*'))


**************** PCN DataBase INF ****************
	* USER: PCN_TEST
	* SID: 172.17.0.2:1521/LHR11G
************ Successfully connected! *************
